**This requires > the standard 2GB of memory in docker. It worked with 3.5GB, but less may be possible.**

In [1]:
import pandas as pd
from ebmdatalab import bq
from lib.outliers import *

In [2]:
with open("../data/static_outlier_sql/chem_per_para.sql") as sql:
    query = sql.read()
#chem_per_para = bq.cached_read(query, csv_path='data/chem_per_para.zip')

## reload specifying data type currently required
## due to https://github.com/ebmdatalab/datalab-pandas/issues/26
chem_per_para = pd.read_csv('../data/chem_per_para.zip',dtype={'subpara': str})
chem_per_para.head()

,practice,pcn,ccg,chemical,subpara,numerator
0,C83009,U58704,04D,1001040C0,1001040,712
1,H81069,U67032,10C,0601021M0,0601021,406
2,M83096,U91937,05G,0307000J0,0307000,302
3,A83030,U01989,00J,0202020D0,0202020,148
4,N81117,U85986,02F,0906026M0,0906026,235


In [3]:
## WHAT TO DO WHERE DENOMINATOR == 0?

stats_class = StaticOutlierStats(
    df=chem_per_para,
    entity_type='ccg',
    num_code='chemical',
    denom_code='subpara'
)
stats = stats_class.get_table()
stats.head()

In [4]:
from lib.make_html import write_to_template
    
def loop_over_everything(df, entities):
    for ent_type in entities:
        entity_names = entity_names_query(ent_type)
        stats_class = StaticOutlierStats(
            df=df,
            entity_type=ent_type,
            num_code='chemical',
            denom_code='subpara'
        )
        stats = stats_class.get_table()
        
        table_high = create_out_table(
            df=stats,
            attr=stats_class,
            entity_type=ent_type,
            table_length=5,
            ascending=False
        )
        table_low = create_out_table(
            df=stats,
            attr=stats_class,
            entity_type=ent_type,
            table_length=5,
            ascending=True
        )
        
        codes = stats.index.get_level_values(0).unique()[0:10]
        for code in tqdm(codes, desc=f'Writing HTML: {ent_type}'):
            output_file = f'static_{ent_type}_{code}'
            write_to_template(
                entity_names.loc[code,'name'],
                table_high.loc[code],
                table_low.loc[code],
                output_file,
            )

In [5]:
loop_over_everything(chem_per_para, ['ccg'])

/root/.pyenv/versions/3.8.1/lib/python3.8/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


/root/.pyenv/versions/3.8.1/lib/python3.8/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
